#  Gradio 챗봇 구현 (간단한 QA 애플리케이션)

### **학습 목표:** LangChain의 LCEL을 활용하여 Gradio 기반의 AI 챗봇을 설계한다.

---

##  환경 설정

In [22]:
# langsmith -> langfuse 공통 코드

# ========== Langfuse 공통 설정 (한 번만 추가) ==========
from langfuse.langchain import CallbackHandler
from dotenv import load_dotenv
from langchain_core.runnables.base import Runnable
import functools

# 환경변수 로드
load_dotenv()

# 전역 Langfuse handler 생성
_langfuse_handler = CallbackHandler()

# 기존 invoke 메서드를 래핑
_original_invoke = Runnable.invoke

@functools.wraps(_original_invoke)
def _invoke_with_langfuse(self, input, config=None, **kwargs):
    if config is None:
        config = {}
    if "callbacks" not in config:
        config["callbacks"] = []
    config["callbacks"].append(_langfuse_handler)
    return _original_invoke(self, input, config, **kwargs)

# Monkey patch 적용
Runnable.invoke = _invoke_with_langfuse



In [23]:
from dotenv import load_dotenv
load_dotenv()

True

## Simple QA Chain  

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 파이썬(Python) 코드 작성을 도와주는 AI 어시스턴트입니다."),
    ("human", "{user_input}")
])

# LLM 모델 정의
model = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.3, 
    )

# 프롬프트 템플릿 + LLM 모델 + 출력파서를 연결하여 체인 생성
chain = prompt | model | StrOutputParser()

# 체인 실행
response = chain.invoke({
    "user_input": "파이썬에서 리스트를 정렬하는 방법은 무엇인가요?"
})

# AI의 응답 텍스트를 출력 
print(response)

파이썬에서 리스트를 정렬하는 방법은 여러 가지가 있습니다. 대표적으로 두 가지 방법이 있습니다:

1. **리스트의 `sort()` 메서드 사용하기**  
   - 리스트 자체를 **제자리(in-place)**에서 정렬합니다.  
   - 반환값은 `None`입니다.  
   - 기본적으로 오름차순으로 정렬합니다.

   ```python
   numbers = [5, 2, 9, 1]
   numbers.sort()
   print(numbers)  # 출력: [1, 2, 5, 9]
   ```

   - 내림차순으로 정렬하려면 `reverse=True` 옵션을 사용합니다.

   ```python
   numbers = [5, 2, 9, 1]
   numbers.sort(reverse=True)
   print(numbers)  # 출력: [9, 5, 2, 1]
   ```

2. **내장 함수 `sorted()` 사용하기**  
   - 원본 리스트는 변경하지 않고, 정렬된 새로운 리스트를 반환합니다.

   ```python
   numbers = [5, 2, 9, 1]
   sorted_numbers = sorted(numbers)
   print(sorted_numbers)  # 출력: [1, 2, 5, 9]
   print(numbers)         # 원본 리스트는 그대로: [5, 2, 9, 1]
   ```

   - `sorted()`도 `reverse=True` 옵션을 지원합니다.

   ```python
   numbers = [5, 2, 9, 1]
   sorted_numbers = sorted(numbers, reverse=True)
   print(sorted_numbers)  # 출력: [9, 5, 2, 1]
   ```

---

### 추가 팁: 정렬 기준 지정하기 (`key` 매개변수)

정렬할 때 특정 기준을 지정할 수도 있습니다.

```python
words = ['apple', 'banana', 'cherry', 'd

In [25]:
# 마크다운 출력
from IPython.display import display, Markdown

display(Markdown(response))

파이썬에서 리스트를 정렬하는 방법은 여러 가지가 있습니다. 대표적으로 두 가지 방법이 있습니다:

1. **리스트의 `sort()` 메서드 사용하기**  
   - 리스트 자체를 **제자리(in-place)**에서 정렬합니다.  
   - 반환값은 `None`입니다.  
   - 기본적으로 오름차순으로 정렬합니다.

   ```python
   numbers = [5, 2, 9, 1]
   numbers.sort()
   print(numbers)  # 출력: [1, 2, 5, 9]
   ```

   - 내림차순으로 정렬하려면 `reverse=True` 옵션을 사용합니다.

   ```python
   numbers = [5, 2, 9, 1]
   numbers.sort(reverse=True)
   print(numbers)  # 출력: [9, 5, 2, 1]
   ```

2. **내장 함수 `sorted()` 사용하기**  
   - 원본 리스트는 변경하지 않고, 정렬된 새로운 리스트를 반환합니다.

   ```python
   numbers = [5, 2, 9, 1]
   sorted_numbers = sorted(numbers)
   print(sorted_numbers)  # 출력: [1, 2, 5, 9]
   print(numbers)         # 원본 리스트는 그대로: [5, 2, 9, 1]
   ```

   - `sorted()`도 `reverse=True` 옵션을 지원합니다.

   ```python
   numbers = [5, 2, 9, 1]
   sorted_numbers = sorted(numbers, reverse=True)
   print(sorted_numbers)  # 출력: [9, 5, 2, 1]
   ```

---

### 추가 팁: 정렬 기준 지정하기 (`key` 매개변수)

정렬할 때 특정 기준을 지정할 수도 있습니다.

```python
words = ['apple', 'banana', 'cherry', 'date']
# 단어 길이 기준으로 정렬
words.sort(key=len)
print(words)  # 출력: ['date', 'apple', 'banana', 'cherry']
```

`sorted()`도 동일하게 `key`를 사용할 수 있습니다.

필요하면 더 자세한 예시도 알려드릴 수 있습니다!

## Gradio ChatInterface  
- 설치: pip install gradio --upgrade , uv add gradio

### 1) 기본 구조

In [26]:
import gradio as gr

# 챗봇 함수 정의
def chat_function(message, history):
    # 메시지 출력
    print(f"입력 메시지: {message}")
    print("-"*40)
    # 채팅 히스토리 출력
    print(f"채팅 히스토리:")
    for chat in history:
        print(f"사용자: {chat['role']}, 메시지: {chat['content']}")
        print("-"*40)
    return "응답 메시지"

# 챗봇 인터페이스 생성
demo = gr.ChatInterface(
    fn=chat_function,  # 실행할 함수
    analytics_enabled=False,  # 사용 정보 제공 여부
    type="messages",     # 채팅 기록이 딕셔너리 리스트 형태의 OpenAI 스타일 포맷으로 전달
)

# 챗봇 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


입력 메시지: 1
----------------------------------------
채팅 히스토리:


In [27]:
# 인터페이스 종료
demo.close()

Closing server running on port: 7861


### 2) 간단한 예제: Echo 챗봇

In [28]:
def echo_bot(message, history):
    return f"당신이 입력한 메시지: {message}"

demo = gr.ChatInterface(
    fn=echo_bot,
    title="Echo 챗봇",
    description="입력한 메시지를 그대로 되돌려주는 챗봇입니다.",
    analytics_enabled=False,  
)

demo.launch()

h:\miniconda3\envs\modu\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [29]:
demo.close()

Closing server running on port: 7861


### 3) 스트리밍 응답

In [30]:
# 스트리밍 챗봇 함수 정의
import time

def streaming_bot(message, history):
    response = f"처리 중인 메시지: {message}"
    for i in range(len(response)):
        time.sleep(0.1)          # 0.1초 대기
        yield response[:i+1]

In [31]:
# 스트리밍 챗봇 인터페이스 생성
demo = gr.ChatInterface(
    fn=streaming_bot,
    title="스트리밍 챗봇",
    description="입력한 메시지를 한 글자씩 처리하는 챗봇입니다.",
    analytics_enabled=False,  
)

# 스트리밍 챗봇 인터페이스 실행
demo.launch()

h:\miniconda3\envs\modu\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [32]:
demo.close()

Closing server running on port: 7861


### 4) 추가 입력 컴포넌트
- 최대 응답 길이 등 기타 설정을 위한 추가 입력

In [33]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 파이썬(Python) 코드 작성을 도와주는 AI 어시스턴트입니다."),
    ("human", "{user_input}")
])


# 챗봇 함수 정의
def chat_function(message, history, model, temperature):

    if model == "gpt-4.1-mini":
        model = ChatOpenAI(model=model, temperature=temperature)
    elif model == "gemini-2.0-flash":
        model = ChatGoogleGenerativeAI(model=model, temperature=temperature)

    chain = prompt | model | StrOutputParser()

    response = chain.invoke({
        "user_input": message
    })
    return response

# 챗봇 인터페이스 생성
with gr.Blocks() as demo:
    model_selector = gr.Dropdown(["gpt-4.1-mini", "gemini-2.0-flash"], label="모델 선택")
    slider = gr.Slider(0.0, 1.0, label="Temperature", value=0.3, step=0.1, render=False)   

    gr.ChatInterface(
        fn=chat_function, 
        additional_inputs=[model_selector, slider],
        analytics_enabled=False,  
    )

# 챗봇 인터페이스 실행
demo.launch()

h:\miniconda3\envs\modu\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [13]:
demo.close()

Closing server running on port: 7860


### 5) 예시 질문 설정

In [34]:
# 스트리밍 챗봇 인터페이스 생성
demo = gr.ChatInterface(
    fn=streaming_bot,
    title="스트리밍 챗봇",
    description="입력한 메시지를 한 글자씩 처리하는 챗봇입니다.",
    analytics_enabled=False,  
    examples=[
        "파이썬 코드를 작성하는 방법을 알려주세요",
        "파이썬에서 리스트를 정렬하는 방법은 무엇인가요?",
    ]    
)

# 스트리밍 챗봇 인터페이스 실행
demo.launch()

h:\miniconda3\envs\modu\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [15]:
demo.close()    

Closing server running on port: 7860


### 6) 멀티모달 기능
- `multimodal=True` 옵션
- 이미지나 파일을 처리할 수 있는 멀티모달 챗봇 구현

- message 파라미터:
    ```python
    {
        "text": "user input", 
        "files": [
            "updated_file_1_path.ext",
            "updated_file_2_path.ext", 
            ...
        ]
    }
    ```
- history 파라미터:
    ```python
    [
        {"role": "user", "content": ("cat1.png")},
        {"role": "user", "content": ("cat2.png")},
        {"role": "user", "content": "What's the difference between these two images?"},
    ]
    ```

In [ ]:
import gradio as gr
import base64
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

def convert_to_url(image_path):
    """이미지를 URL 형식으로 변환"""
    with open(image_path, "rb") as image_file:
        # 이미지를 base64로 인코딩
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        return f"data:image/jpeg;base64,{encoded_string}"

def multimodal_bot(message, history):

    model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    
    if isinstance(message, dict):
        # 텍스트와 파일 추출
        text = message.get("text", "")
        
        # 히스토리와 현재 메시지에서 모든 파일 경로 추출
        filepath_list = []
        
        # 히스토리에서 이미지 파일 추출
        print("History:", history)  # 디버깅용
        for exchange in history:
            user_message = exchange[0]
            if isinstance(user_message, tuple):  # 이미지 메시지 확인
                filepath_list.append(user_message[0])
        
        # 현재 메시지의 파일들도 추가
        files = message.get("files", [])
        filepath_list.extend(files)
        
        print("Filepath list:", filepath_list)  # 디버깅용
        
        if filepath_list:
            # 모든 이미지 처리
            image_urls = []
            for file_path in filepath_list:
                try:
                    image_url = convert_to_url(file_path)
                    image_urls.append({"type": "image_url", "image_url": image_url})
                except Exception as e:
                    print(f"이미지 처리 중 오류 발생: {e}")
                    continue
            
            if not image_urls:
                return "이미지 처리 중 오류가 발생했습니다."
            
            # 메시지 구성
            content = [
                {"type": "text", "text": text if text else "이 이미지들에 대해 설명해주세요."},
                *image_urls
            ]
            
            try:
                # API 호출
                response = model.invoke([
                    HumanMessage(content=content)
                ])
                return response.content
            except Exception as e:
                return f"모델 응답 생성 중 오류가 발생했습니다: {str(e)}"
        
        return text if text else "이미지를 업로드해주세요."
    
    return "텍스트나 이미지를 입력해주세요."

# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=multimodal_bot,
    type="messages",
    multimodal=True,
    title="멀티모달 챗봇",
    description="텍스트와 이미지를 함께 처리할 수 있는 챗봇입니다. 이전 대화의 이미지들도 함께 고려합니다.",
    analytics_enabled=False,  
    textbox=gr.MultimodalTextbox(placeholder="텍스트를 입력하거나 이미지를 업로드해주세요.", file_count="multiple", file_types=["image"]),
)

# 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


History: []
Filepath list: ['C:\\Users\\kaydash\\AppData\\Local\\Temp\\gradio\\b52737409f5f42b5816084b3dbd64b4985c8f8b3e29b18c3f8675cd8cf68dac9\\2021_07_14 오후 1_29 Office Lens.jpg']
History: []
Filepath list: ['C:\\Users\\kaydash\\AppData\\Local\\Temp\\gradio\\8ab21706330eabd4676a0463729c02feed683691b6cf171a136e48b93b06d2fc\\2021_07_14 오후 1_28 Office Lens.jpg']


In [37]:
demo.close()

Closing server running on port: 7864


### 7) PDF 뷰어
- 설치: pip install gradio_pdf 또는 poetry add gradio_pdf

In [38]:
from gradio_pdf import PDF

def answer_invoke(message, history):   
    return message

with gr.Blocks(
    analytics_enabled=False,  
) as demo:
    with gr.Row():
        # API Key Section
        api_key_input = gr.Textbox(
            label="Enter OpenAI API Key",
            type="password",
            placeholder="sk-..."
        )
        
    with gr.Row():
        # PDF Upload and Chat Interface
        with gr.Column(scale=2):
            pdf_file = PDF(
                label="Upload PDF File",
                height=600,  # PDF 뷰어 높이 설정
            )
        with gr.Column(scale=1):
            chatbot = gr.ChatInterface(
                fn=answer_invoke,
                type="messages",
                title="PDF-based Chatbot",
                description="Upload a PDF file and ask questions about its contents.",
            )


demo.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [39]:
demo.close()

Closing server running on port: 7864


## Memory 추가

In [40]:
# chat_history 플레이스홀더를 사용
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# 메시지 플레이스홀더가 있는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 파이썬(Python) 코드 작성을 도와주는 AI 어시스턴트입니다."),
    MessagesPlaceholder("chat_history"),
    ("system", "이전 대화 내용을 참고하여 질문에 대해서 친절하게 답변합니다."),
    ("human", "{user_input}")
])

# 프롬프트 템플릿 + LLM 모델 + 출력파서를 연결하여 체인 생성
chain = prompt | model | StrOutputParser()


# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)
def answer_invoke(message, history):

    history_messages = []
    for msg in history:
        if msg['role'] == "user":
            history_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_messages.append(AIMessage(content=msg['content']))

    history_messages.append(HumanMessage(content=message))
    response = chain.invoke({
        "chat_history": history_messages,
        "user_input": message
    })
    return response
    

# Gradio ChatInterface 객체 생성
demo = gr.ChatInterface(
    fn=answer_invoke,         # 메시지 처리 함수
    type="messages",
    title="파이썬 코드 어시스턴트", # 채팅 인터페이스의 제목
    )

# Gradio 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [41]:
# Gradio 인터페이스 종료
demo.close()

Closing server running on port: 7864


# [실습 프로젝트]

- **다음과 같은 요구사항을 Gradio ChatInterface로 구현합니다**

- 주제: 맞춤형 여행 일정 계획 어시스턴트
- 기능: 
   - OpenAI Chat Completion API와 LangChain을 활용하여 사용자의 선호도에 맞는 여행 일정을 생성
   - LCEL을 사용하여 단계별 프롬프트 체인 구성 (사용자 입력 분석 -> 일정 생성 -> 세부 계획 수립)
   - 채팅 히스토리 사용하여 답변 생성
   - Gradio 인터페이스를 통해 사용자와 대화형으로 상호작용

- 주요 포인트:

   1. **모델 매개변수 최적화**
      - temperature=0.7: 적당한 창의성을 유지하면서 일관된 응답 생성
      - top_p=0.9: 높은 확률의 토큰만 선택하여 응답의 품질 향상
      - presence_penalty와 frequency_penalty: 반복적인 응답을 줄이고 다양한 제안 생성

   2. **시스템 프롬프트 설계**
      - 여행 플래너로서의 역할과 응답 가이드라인을 명확히 정의
      - 구체적인 정보를 포함하도록 지시
      - 한국어 응답 명시

   3. **메모리 관리**
      - Gradio 또는 LangChain 메모리 기능을 사용하여 대화 컨텍스트 유지
      - 이전 대화 내용을 바탕으로 연속성 있는 응답 생성

[예시 답안]

In [42]:
# 필수 라이브러리 임포트
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import gradio as gr

# 메시지 플레이스홀더가 있는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "여행 일정을 계획해주는 AI 어시스턴트입니다."),
    MessagesPlaceholder("chat_history"),
    ("system", "사용자의 선호도에 맞는 여행 일정을 생성합니다."),
    ("human", "{user_input}")
])

# LLM 모델 정의
model = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.7, 
    top_p=0.9,
    presence_penalty=0.3,
    frequency_penalty=0.3,
    )

# 프롬프트 템플릿 + LLM 모델 + 출력파서를 연결하여 체인 생성
chain = prompt | model | StrOutputParser()

# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)
def answer_invoke(message, history):

    history_messages = []
    for msg in history:
        if msg['role'] == "user":
            history_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_messages.append(AIMessage(content=msg['content']))

    history_messages.append(HumanMessage(content=message))
    response = chain.invoke({
        "chat_history": history_messages,
        "user_input": message
    })
    return response

# Gradio ChatInterface 객체 생성
demo = gr.ChatInterface(
    fn=answer_invoke,         # 메시지 처리 함수
    type="messages",
    title="맞춤형 여행 일정 계획 어시스턴트", # 채팅 인터페이스의 제목
    )

# Gradio 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


입력 메시지: 포항 2박 3일 + 강릉 1박2일 + 부산 1박 2일 + 양평 1박 2일 + 제주도 2박 3일 계획을 표로 짜줘
----------------------------------------
채팅 히스토리:
입력 메시지: 안녕?
----------------------------------------
채팅 히스토리:
입력 메시지: 안녕?
----------------------------------------
채팅 히스토리:
입력 메시지: 안녕
----------------------------------------
채팅 히스토리:
입력 메시지: 안녕
----------------------------------------
채팅 히스토리:


In [ ]:
# Gradio 인터페이스 종료
demo.close()